# RecommendKit Quickstart Guide

This notebook demonstrates how to use RecommendKit to build a recommendation system from scratch.

## What You'll Learn
1. Loading and exploring recommendation data
2. Understanding user and item features
3. Configuring the model with SimpleFusion
4. Training a recommendation model
5. Running inference to get recommendations


## 1. Setup and Imports


In [1]:
import sys
import os
import json
import torch
import numpy as np
from pathlib import Path

# Add project root to path
project_root = Path.cwd()
sys.path.append(str(project_root))

# Import RecommendKit components
from input_processor import Inputs
from trainer.pipeline_builder import RecommendationPipeline, save_complete_model
from trainer.data_loader import create_data_loaders, load_interactions_from_input
from trainer.trainer import train_model

print("✅ All imports successful!")


✅ All imports successful!


## 2. Load and Explore Data

RecommendKit uses a simple JSON format for data. Let's load and explore the dataset.


In [2]:
# Load data
data_path = "datasets/synthetic/correlated_dataset.json"

print("🔄 Loading data...")
inputs = Inputs()
inputs.configure_validators(image_check_files=False)  # Skip image file validation for demo
result = inputs.load_from_json(data_path)

if not result.is_valid:
    print("❌ Data loading errors:")
    for error in result.errors:
        print(f"  - {error}")
else:
    print("✅ Data loaded successfully!")
    
    # Get data
    user_data = inputs.get_user_data()
    item_data = inputs.get_item_data()
    
    print(f"\n📊 Dataset Statistics:")
    print(f"   Users: {len(user_data)}")
    print(f"   Items: {len(item_data)}")


🔄 Loading data...
Loaded 13600 interactions
✅ Data loaded successfully!

📊 Dataset Statistics:
   Users: 1000
   Items: 100


### 2.1 Explore User Features

User features represent characteristics of users in the system. Let's examine a sample user:


In [3]:
# Display first user as example
sample_user = user_data[0]

print("👤 Sample User Features:")
print(f"   User ID: {sample_user['user_id']}")
print(f"\n   Categorical Features (discrete values):")
for key, value in sample_user.get('categorical', {}).items():
    print(f"     - {key}: {value}")
    
print(f"\n   Continuous Features (numeric values):")
for key, value in sample_user.get('continuous', {}).items():
    print(f"     - {key}: {value}")
    
print(f"\n   Temporal Features (sequential data):")
if 'temporal' in sample_user:
    prev_items = sample_user['temporal'].get('previous_liked_items', [])
    print(f"     - Previous liked items: {len(prev_items)} items")
    print(f"     - Sample items: {prev_items[:5]}...")

print("\n" + "="*60)
print("\n💡 Feature Types Explained:")
print("   • Categorical: Discrete values (e.g., occupation, location)")
print("   • Continuous: Numeric values (e.g., age, salary)")
print("   • Temporal: Sequential interaction history")


👤 Sample User Features:
   User ID: 1

   Categorical Features (discrete values):
     - occupation: scientist
     - location: Canada

   Continuous Features (numeric values):
     - age: 41.0
     - salary: 121172.52

   Temporal Features (sequential data):
     - Previous liked items: 14 items
     - Sample items: [83, 84, 85, 86, 87]...


💡 Feature Types Explained:
   • Categorical: Discrete values (e.g., occupation, location)
   • Continuous: Numeric values (e.g., age, salary)
   • Temporal: Sequential interaction history


### 2.2 Explore Item Features

Item features represent characteristics of items/products. Let's examine a sample item:


In [4]:
# Display first item as example
sample_item = item_data[0]

print("📦 Sample Item Features:")
print(f"   Item ID: {sample_item['item_id']}")
print(f"\n   Categorical Features:")
for key, value in sample_item.get('categorical', {}).items():
    print(f"     - {key}: {value}")
    
print(f"\n   Continuous Features:")
for key, value in sample_item.get('continuous', {}).items():
    print(f"     - {key}: {value}")
    
print(f"\n   Text Features:")
for key, value in sample_item.get('text', {}).items():
    print(f"     - {key}: {value}")

print("\n" + "="*60)
print("\n💡 Item Feature Types:")
print("   • Categorical: Item categories, brands, etc.")
print("   • Continuous: Price, rating, etc.")
print("   • Text: Name, description, etc.")


📦 Sample Item Features:
   Item ID: 1

   Categorical Features:
     - category: tech

   Continuous Features:
     - price: 1350.97

   Text Features:
     - name: Laptop
     - description: High-quality Laptop for professionals and enthusiasts


💡 Item Feature Types:
   • Categorical: Item categories, brands, etc.
   • Continuous: Price, rating, etc.
   • Text: Name, description, etc.


## 3. Configure the Model

RecommendKit uses JSON configuration files to define model architecture. Let's load and explain the SimpleFusion configuration:


In [5]:
# Load configuration
config_path = "configs/correlated_dataset_config.json"

with open(config_path, 'r') as f:
    config = json.load(f)

print("⚙️  Model Configuration (SimpleFusion):")
print("\n" + "="*60)
print("\n🎯 Core Settings:")
print(f"   • Embedding Dimension: {config['embedding_dim']}")
print(f"   • Loss Type: {config['loss_type']}")

print("\n🔧 Fusion Architecture:")
print(f"   • User SimpleFusion: {config['user_use_simple_fusion']}")
print(f"   • Item SimpleFusion: {config['item_use_simple_fusion']}")
print(f"   • Interaction SimpleFusion: {config['interaction_use_simple_fusion']}")
print("\n   💡 SimpleFusion: Fast concatenation + MLP approach (no transformers)")

print("\n📚 Encoder Configurations:")
print(f"   • Text Encoder: {config['text_encoder_config']['model_name']}")
print(f"   • Categorical Encoder: Hash-based embeddings")
print(f"   • Continuous Encoder: MLP with normalization")
print(f"   • Temporal Encoder: LSTM for sequential data")

print("\n🎓 Training Settings:")
print(f"   • Epochs: {config['num_epochs']}")
print(f"   • Batch Size: {config['batch_size']}")
print(f"   • Learning Rate: {config['learning_rate']}")
print(f"   • Train/Val Split: {config['train_split']}")


⚙️  Model Configuration (SimpleFusion):


🎯 Core Settings:
   • Embedding Dimension: 64
   • Loss Type: bce

🔧 Fusion Architecture:
   • User SimpleFusion: True
   • Item SimpleFusion: True
   • Interaction SimpleFusion: True

   💡 SimpleFusion: Fast concatenation + MLP approach (no transformers)

📚 Encoder Configurations:
   • Text Encoder: glove-wiki-gigaword-50
   • Categorical Encoder: Hash-based embeddings
   • Continuous Encoder: MLP with normalization
   • Temporal Encoder: LSTM for sequential data

🎓 Training Settings:
   • Epochs: 25
   • Batch Size: 32
   • Learning Rate: 0.001
   • Train/Val Split: 0.8


## 4. Create the Model

Now let's create the recommendation model using our configuration:


In [11]:
!python3 -m pip install --break-system-packages gensim

  Using cached scipy-1.16.3-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (62 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 42.7 MB/s eta 0:00:0000:0100:01
Using cached scipy-1.16.3-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (35.7 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.5/121.5 kB 20.4 MB/s eta 0:00:00


In [12]:
print("🔄 Creating model...")

model = RecommendationPipeline(
    embedding_dim=config.get('embedding_dim', 64),
    loss_type=config.get('loss_type', 'bce'),
    
    # User tower configuration
    user_num_attention_layers=config.get('user_num_attention_layers', 0),
    user_num_heads=config.get('user_num_heads', 1),
    user_dropout=config.get('user_dropout', 0.0),
    user_use_cls_token=config.get('user_use_cls_token', False),
    user_use_layer_norm=config.get('user_use_layer_norm', False),
    user_use_simple_fusion=config.get('user_use_simple_fusion', True),
    
    # Item tower configuration
    item_num_attention_layers=config.get('item_num_attention_layers', 0),
    item_num_heads=config.get('item_num_heads', 1),
    item_dropout=config.get('item_dropout', 0.0),
    item_use_simple_fusion=config.get('item_use_simple_fusion', True),
    
    # Interaction modeling configuration
    interaction_num_attention_layers=config.get('interaction_num_attention_layers', 0),
    interaction_num_heads=config.get('interaction_num_heads', 1),
    interaction_dropout=config.get('interaction_dropout', 0.0),
    interaction_use_simple_fusion=config.get('interaction_use_simple_fusion', True),
    
    # Classifier configuration
    classifier_hidden_dims=config.get('classifier_hidden_dims', [64]),
    classifier_dropout=config.get('classifier_dropout', 0.0),
    
    # Encoder configurations
    image_encoder_config=config.get('image_encoder_config'),
    text_encoder_config=config.get('text_encoder_config'),
    categorical_encoder_config=config.get('categorical_encoder_config'),
    continuous_encoder_config=config.get('continuous_encoder_config'),
    temporal_encoder_config=config.get('temporal_encoder_config'),
    
    # Item data for temporal encoder
    item_data=item_data
)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"✅ Model created successfully!")
print(f"   Total parameters: {total_params:,}")
print(f"   Trainable parameters: {trainable_params:,}")
print(f"   Device: {'cuda' if torch.cuda.is_available() else 'cpu'}")


🔄 Creating model...
Loading Word2Vec model: glove-wiki-gigaword-50
Successfully loaded Word2Vec model with 50 dimensions
Temporal encoder: Creating with item lookup enabled
✅ Model created successfully!
   Total parameters: 975,361
   Trainable parameters: 975,361
   Device: cpu


## 5. Prepare Data Loaders

We need to create data loaders for training. RecommendKit automatically handles positive/negative sampling:


In [13]:
print("🔄 Loading interactions...")
interactions = load_interactions_from_input(inputs)
print(f"✅ Loaded {len(interactions)} interactions")

print("\n🔄 Creating data loaders...")
train_loader, val_loader = create_data_loaders(
    inputs=inputs,
    interactions=interactions,
    train_split=config.get('train_split', 0.8),
    batch_size=config.get('batch_size', 32),
    negative_sampling_ratio=config.get('negative_sampling_ratio', 1.0),
    seed=config.get('seed', 42)
)

print(f"✅ Data loaders created!")
print(f"   Train batches: {len(train_loader)}")
print(f"   Validation batches: {len(val_loader)}")
print(f"\n💡 Negative sampling: For each positive interaction, we generate")
print(f"   {config.get('negative_sampling_ratio', 1.0)} negative samples (items user hasn't interacted with)")


🔄 Loading interactions...
Loaded 13600 real interactions from input data
Positive interactions: 13600
Negative interactions: 0
Positive ratio: 1.000
✅ Loaded 13600 interactions

🔄 Creating data loaders...
Split interactions: 10880 train, 2720 validation
Created dataset with 10880 positive and 10880 negative samples
Created dataset with 2720 positive and 2720 negative samples
✅ Data loaders created!
   Train batches: 680
   Validation batches: 170

💡 Negative sampling: For each positive interaction, we generate
   1.0 negative samples (items user hasn't interacted with)


## 6. Train the Model

Now let's train the model. We'll use fewer epochs for this quickstart:


In [14]:
# Use fewer epochs for quickstart
num_epochs = 3  # Reduced from config for faster demo

print(f"🚀 Starting training for {num_epochs} epochs...")
print("="*60)

history = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=num_epochs,
    learning_rate=config.get('learning_rate', 0.001),
    optimizer_type=config.get('optimizer_type', 'adam'),
    scheduler_type=config.get('scheduler_type', 'cosine'),
    print_every=config.get('print_every', 1)
)

print("\n" + "="*60)
print("✅ Training completed!")
print(f"\n📊 Final Results:")
print(f"   Train Loss: {history['train_losses'][-1]:.4f}")
print(f"   Train Accuracy: {history['train_accuracies'][-1]:.4f}")
if history['val_losses']:
    print(f"   Val Loss: {history['val_losses'][-1]:.4f}")
    print(f"   Val Accuracy: {history['val_accuracies'][-1]:.4f}")


🚀 Starting training for 3 epochs...
Training on cpu
Model parameters: 975,361
Starting training for 3 epochs



Epoch 1/3 Summary (15.2m)
  Train Loss: 0.4773, Train Acc: 0.7485
  Val Loss: 0.3318, Val Acc: 0.8713
  LR: 0.000750




Epoch 2/3 Summary (28.6m)
  Train Loss: 0.2259, Train Acc: 0.9196
  Val Loss: 0.2635, Val Acc: 0.9017
  LR: 0.000250




Epoch 3/3 Summary (42.6m)
  Train Loss: 0.1827, Train Acc: 0.9349
  Val Loss: 0.2530, Val Acc: 0.9114
  LR: 0.000000


Training completed in 42.6 minutes

✅ Training completed!

📊 Final Results:
   Train Loss: 0.1827
   Train Accuracy: 0.9349
   Val Loss: 0.2530
   Val Accuracy: 0.9114


## 7. Save the Model

Let's save the trained model for later use:


In [15]:
save_dir = "models"
model_name = "quickstart_model"

print(f"💾 Saving model to {save_dir}/{model_name}...")
save_complete_model(model, save_dir, model_name)
print("✅ Model saved successfully!")


💾 Saving model to models/quickstart_model...
✅ Complete model saved to models/
   Main model: models/quickstart_model.pt
   Config: models/quickstart_model_config.json
   Manifest: models/quickstart_model_manifest.json
   Components: 8 individual state dicts
✅ Model saved successfully!


## 8. Run Inference

Now let's use the trained model to generate recommendations for a user:


In [17]:
# Set model to evaluation mode
model.eval()

# Select a user for recommendations
target_user_id = 2
k = 10  # Number of recommendations

# Find user data
target_user = None
for user in user_data:
    if user['user_id'] == target_user_id:
        target_user = user
        break

if target_user is None:
    print(f"❌ User {target_user_id} not found!")
else:
    print(f"👤 Generating recommendations for User {target_user_id}")
    print(f"   Occupation: {target_user.get('categorical', {}).get('occupation', 'N/A')}")
    print(f"   Location: {target_user.get('categorical', {}).get('location', 'N/A')}")
    print(f"   Age: {target_user.get('continuous', {}).get('age', 'N/A')}")
    print("\n🔄 Computing recommendations...")
    
    # Prepare user data for model
    user_features = {
        'categorical': target_user.get('categorical', {}),
        'continuous': target_user.get('continuous', {}),
        'temporal': target_user.get('temporal', {})
    }
    
    # Compute similarities with all items
    similarities = []
    
    with torch.no_grad():
        for item in item_data:
            item_features = {
                'categorical': item.get('categorical', {}),
                'continuous': item.get('continuous', {}),
                'text': item.get('text', {})
            }
            
            # Get prediction score
            logits = model(
                user_data={k: v for k, v in user_features.items() if v},
                item_data={k: v for k, v in item_features.items() if v}
            )
            
            # Convert to probability
            score = torch.sigmoid(logits).item()
            
            similarities.append({
                'item_id': item['item_id'],
                'item': item,
                'score': score
            })
    
    # Sort by score and get top-k
    similarities.sort(key=lambda x: x['score'], reverse=True)
    top_k = similarities[:k]
    
    print(f"\n🎯 Top {k} Recommendations for User {target_user_id}")
    print("="*60)
    
    for i, rec in enumerate(top_k, 1):
        item = rec['item']
        item_name = item.get('text', {}).get('name', f"Item {item['item_id']}")
        print(f"\n{i}. {item_name}")
        print(f"   Score: {rec['score']:.4f}")
        print(f"   Item ID: {item['item_id']}")
        
        if 'categorical' in item and item['categorical']:
            cat_info = ", ".join([f"{k}: {v}" for k, v in item['categorical'].items()])
            print(f"   Categories: {cat_info}")
        
        if 'continuous' in item and item['continuous']:
            cont_info = ", ".join([f"{k}: {v}" for k, v in item['continuous'].items()])
            print(f"   Features: {cont_info}")


👤 Generating recommendations for User 2
   Occupation: lawyer
   Location: India
   Age: 51.0

🔄 Computing recommendations...



🎯 Top 10 Recommendations for User 2

1. Business Suit
   Score: 0.9698
   Item ID: 59
   Categories: category: business
   Features: price: 214.38

2. Office Chair
   Score: 0.9689
   Item ID: 66
   Categories: category: business
   Features: price: 643.2

3. Briefcase
   Score: 0.9688
   Item ID: 60
   Categories: category: business
   Features: price: 365.62

4. Tie
   Score: 0.9685
   Item ID: 62
   Categories: category: business
   Features: price: 548.0

5. Document Organizer
   Score: 0.9680
   Item ID: 63
   Categories: category: business
   Features: price: 489.86

6. Desk Lamp
   Score: 0.9678
   Item ID: 65
   Categories: category: business
   Features: price: 690.0

7. Leather Shoes
   Score: 0.9676
   Item ID: 61
   Categories: category: business
   Features: price: 792.67

8. File Cabinet
   Score: 0.9674
   Item ID: 64
   Categories: category: business
   Features: price: 579.23

9. Sunglasses
   Score: 0.7941
   Item ID: 95
   Categories: category: weather_hot
   Featur

## 9. Next Steps

Congratulations! You've successfully:
1. ✅ Loaded and explored recommendation data
2. ✅ Configured a SimpleFusion model
3. ✅ Trained a recommendation model
4. ✅ Generated personalized recommendations

### What's Next?

- **Try Attention-Based Fusion**: Use `correlated_dataset_attention_config.json` for transformer-based feature fusion
- **Experiment with Configurations**: Adjust embedding dimensions, hidden layers, and other hyperparameters
- **Use Your Own Data**: Format your data in the JSON structure shown above
- **Production Deployment**: Use the saved model with `inference.py` for production inference

### Learn More

Check out the README.md for:
- Detailed architecture explanations
- Advanced customization options
- Integration examples

Happy recommending! 🚀
